In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import string
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec23.txt'

In [2]:
ops = sim.read_ops(INPUT_PATH.read_text())

In [3]:
class IOWrapper:
    def __init__(self, num_comps):
        self.input_wrappers = [InputWrapper(i) for i in range(num_comps)]
        self.output_wrappers = [OutputWrapper(self) for i in range(num_comps)]
        self.nat = NAT(self)
    
    def send(self, to_comp, *msgs):
        if to_comp == 255:
            for msg in msgs:
                self.nat.send(msg)
        else:
            for msg in msgs:
                self.input_wrappers[to_comp].queue.append(msg)

class InputWrapper:
    def __init__(self, val):
        self.queue = deque([val])
        self.times_empty = 0

    def get(self):
        if self.queue:
            self.times_empty = 0
            return self.queue.popleft()
        self.times_empty += 1
        return -1

    
class OutputWrapper:
    def __init__(self, io_wrapper):
        self.io_wrapper = io_wrapper
        self.to_whom = None
        self.output_counter = 0
    
    def __call__(self, val):
        self.output_counter = (self.output_counter + 1) % 3
        if self.output_counter == 1:
            self.to_whom = val
        elif self.output_counter == 2:
            self.io_wrapper.send(self.to_whom, val)
        else:
            self.io_wrapper.send(self.to_whom, val)

            
class Computers:
    def __init__(self, ops, num_comps):
        self.io_wrapper = IOWrapper(num_comps)
        self.computers = []
        for input_wrapper, output_wrapper in zip(self.io_wrapper.input_wrappers, self.io_wrapper.output_wrappers):
            self.computers.append(sim.Computer(
                ops,
                inplace=False,
                inputs=input_wrapper,
                output_func=output_wrapper
            ))
    
    def run(self, step_size=10, stop_early=False):
        done = [False] * len(self.computers)
        while not all(done):
            for i, (is_done, computer) in enumerate(zip(done, self.computers)):
                if not is_done:
                    out = computer.simulate(num_steps=step_size)
                    if out == -1:
                        done[i] = True
            if stop_early and self.io_wrapper.nat.y is not None:
                print(f'The answer to part 1 is {self.io_wrapper.nat.y}')
                break
            
            self.io_wrapper.nat.monitor()
            if self.io_wrapper.nat.part2_done:
                break


class NAT:
    def __init__(self, io_wrapper):
        self.io_wrapper = io_wrapper
        self.x = None
        self.y = None
        self.counter = 0
        self.part2_done = False
        self.last_y = None

    def send(self, val):
        self.counter += 1
        self.counter %= 2
        if self.counter == 1:
            self.x = val
        else:
            self.y = val

    def monitor(self):
        if all(input_wrapper.times_empty > 1 for input_wrapper in self.io_wrapper.input_wrappers):
            if self.last_y == self.y:
                print(f'The answer to part 2 is {self.y}')
                self.part2_done = True
            self.last_y = self.y
            self.io_wrapper.send(0, self.x, self.y)

In [4]:
Computers(ops, 50).run(stop_early=True)

The answer to part 1 is 23259


In [5]:
Computers(ops, 50).run(stop_early=False)

The answer to part 2 is 15742
